# LangChain Tool & Agent Tutorial
This tutorial demonstrates how to:
1. Create custom tools for LLMs
2. Use tools with LLMs directly
3. Build a tool-calling agent with AgentExecutor
4. Build a ReAct agent

## 1. Setup: Loading the Language Model
First, we'll load GPT-4 through the OpenAI API. We use ChatOpenAI because it's better at following instructions
and using tools compared to base chat models.

In [1]:
# Load LLM Models
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

## 2. Creating Custom Tools
Tools in LangChain are functions that LLMs can use to interact with external systems or perform computations.
Each tool should:
- Have clear documentation (docstring)
- Have type hints for inputs and outputs
- Perform a single, well-defined task
Below we create two example tools:
1. A simple multiplication tool
2. A mock temperature lookup tool

In [2]:
from langchain_core.tools import tool

# The @tool decorator automatically converts our Python functions into LangChain tools
# It uses the function's docstring and type hints to tell the LLM how to use the tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


@tool
def gettempature(a: str) -> float:
    """Get the tempature of a city named a in celsius.

    Args:
        a: city name
    """
    return 23.5


# Create a list of available tools
tools = [multiply, gettempature]

# Bind the tools to our LLM
# This creates a new LLM that knows about and can use these tools
llm_with_tools = llm.bind_tools([multiply, gettempature])

#### 2.1 Basic Tool Usage
Let's first see how the LLM behaves when it doesn't need to use any tools.
The LLM should recognize that a simple greeting doesn't require tool usage.

In [3]:
result = llm_with_tools.invoke("Hello world!")
print(result)

content='Hello! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 96, 'total_tokens': 107, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'finish_reason': 'stop', 'logprobs': None} id='run-9529ef0b-08b5-4db7-8830-347ef492ef6a-0' usage_metadata={'input_tokens': 96, 'output_tokens': 11, 'total_tokens': 107, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


#### 2.2 Tool Selection and Usage
Now let's see how the LLM handles a question that could benefit from using a tool.
The LLM should:
1. Recognize that the multiplication tool is relevant
2. Call the tool with appropriate parameters
3. Use the tool's output to form its response
4. Pass the execution result to LLM to form the final response

In [7]:
result = llm_with_tools.invoke("What is 3 times by 2?")
print(result)
# Use the generated JSON schema to execute the tool
multiply.invoke(result.tool_calls[0])

content='' additional_kwargs={'tool_calls': [{'id': 'call_4zP2wXRtPjEsd0Y5SB63pmgr', 'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 102, 'total_tokens': 120, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-75e86024-ff06-48af-bef2-f95249502ae9-0' tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_4zP2wXRtPjEsd0Y5SB63pmgr', 'type': 'tool_call'}] usage_metadata={'input_tokens': 102, 'output_tokens': 18, 'total_tokens': 120, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


ToolMessage(content='6', name='multiply', tool_call_id='call_4zP2wXRtPjEsd0Y5SB63pmgr')

In [8]:
result = llm_with_tools.invoke("What is the temperature at Kent Ridge?")
print(result)
gettempature.invoke(result.tool_calls[0])

content='' additional_kwargs={'tool_calls': [{'id': 'call_B41os198KLSp2MN6EiucPihw', 'function': {'arguments': '{"a":"Kent Ridge"}', 'name': 'gettempature'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 101, 'total_tokens': 118, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-ab4e135f-bdbf-4d8b-a1c7-acffe6a8e10d-0' tool_calls=[{'name': 'gettempature', 'args': {'a': 'Kent Ridge'}, 'id': 'call_B41os198KLSp2MN6EiucPihw', 'type': 'tool_call'}] usage_metadata={'input_tokens': 101, 'output_tokens': 17, 'total_tokens': 118, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


ToolMessage(content='23.5', name='gettempature', tool_call_id='call_B41os198KLSp2MN6EiucPihw')

#### 2.3 Pass the execution result to LLM

Here we'll see how to handle tool calls manually without using LangChain's high-level abstractions.
This helps understand what's happening under the hood.

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

# Example query that should trigger tool usage
query = "What is the temperature at Kent Ridge?"

# Create a conversation with system and user messages
messages = [SystemMessage("You are a helpful assistant"), HumanMessage(query)]

# Get the AI's response, which may include tool calls
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

# Process any tool calls in the response
for tool_call in ai_msg.tool_calls:
    # Map tool names to actual tool functions
    selected_tool = {"multiply": multiply, "gettempature": gettempature}[
        tool_call["name"].lower()
    ]
    # Execute the tool and get its response
    tool_msg = selected_tool.invoke(tool_call)
    # Add the tool's response to the conversation
    messages.append(tool_msg)

# Get the final response incorporating tool results
final_ai_msg = llm_with_tools.invoke(messages)
messages.append(final_ai_msg)

# Print the entire conversation
print("Full conversation with tool usage:")
for message in messages:
    print(message)
    print("---------")

In [ ]:
llm_with_tools.invoke(messages)

## 3. Wrap up: Build tool-calling agent with AgentExecutor

In [10]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Respond only in Korean."),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [11]:
agent_executor.invoke({"input": "What is the tempature of Kent Ridge?"})

{'input': 'What is the tempature of Kent Ridge?',
 'output': '켄트 리지의 온도는 섭씨 23.5도입니다.'}

## 4. Build a ReAct Agent

Here we'll use the ReAct (Reasoning + Acting) agent, which:
1. Thinks about what to do (Reasoning)
2. Takes actions using tools (Acting)
3. Observes results
4. Repeats until the task is complete

In [12]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults

# Set up a search tool for the agent to use
tools = [TavilySearchResults(max_results=1)]
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")
# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

/Users/rz/Documents/projects_code/.venv/lib/python3.12/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
# Run the agent on a complex query that requires:
# 1. Understanding the question
# 2. Searching for relevant information
# 3. Extracting the specific detail requested
agent_executor.invoke(
    {
        "input": "What is the headquarters of the company where the lecturer of BT5153 works? BT5153 is a module offered by NUS, MBSA.?"
    }
)